<a href="https://colab.research.google.com/github/DevendraTomar/llm-learnings/blob/main/GenAI_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GenAI Notebook Overview

This notebook demonstrates the usage of LangChain for interacting with Large Language Models (LLMs), specifically Google's Gemini and Groq's LLaMa models.

**Steps:**

1. **Introduction to GenAI:**
   - Briefly explains the 4 major components of GenAI: Model, Prompt, Output.

2. **LangChain Library:**
   - Introduces LangChain and its benefits for LLM application development:
     - Accelerated Development
     - Scalability and Flexibility
     - Improved Productivity
     - Enhanced User Experiences
     - Future-Proof Investments

3. **Working with Google Gemini:**
   - **Installation:** Installs the `langchain-google-genai` package.
   - **Loading Prompt:** Defines a prompt template for translation using `ChatPromptTemplate`.
   - **Loading Model:** Initializes a `ChatGoogleGenerativeAI` instance with the Gemini model and API key.
   - **Invoking Model:** Sends a prompt for translation and gets the response.
   - **Formatting Response:** Uses Pydantic to structure the response into a `Translation` object.
   - **Loading into Spark:** Creates a Spark DataFrame from the structured response.
   - **Controlling Temperature:** Demonstrates how to adjust the `temperature` parameter for controlling randomness in Gemini's responses.

4. **Working with Groq's LLaMa and Tool Calling:**
   - **Installation:** Installs the `langchain-groq` package.
   - **Loading Model:** Initializes a `ChatGroq` instance with the LLaMa model and API key.
   - **External Functions:** Defines a custom tool function using `@tool` decorator.
   - **Model with Tools:** Binds the tool function to the LLaMa model using `bind_tools`.
   - **Tool Calling:** Demonstrates how to invoke the model with tools and access tool call results.

**Key Takeaways:**

- LangChain simplifies the development of LLM-powered applications.
- It provides a flexible and scalable framework for integrating various LLMs.
- Tool calling enables extending LLM capabilities with external functions.
- Temperature control allows adjusting the randomness of LLM outputs.

## GenAI's 4 Major Components:

1. **Model:** The core algorithm that powers GenAI, responsible for understanding and generating text.
2. **Prompt:** The input given to the model, guiding it to produce the desired output.
3. **Output:** The generated text or other content produced by the model

# LangChain Library

LangChain is a framework for developing applications powered by large language models (LLMs).

| **Benefit**                   | **Description**                                                                                                                                                     |
|-------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Accelerated Development**    | LangChain abstracts the complexities of integrating and managing LLMs, reducing development time and allowing developers to focus on delivering value-added features. |
| **Scalability and Flexibility**| With a modular architecture and support for multiple LLMs, LangChain ensures applications can scale and adapt to evolving language models.                           |
| **Improved Productivity**      | By handling LLM integration and memory management, LangChain allows developers to focus on innovative solutions, boosting overall productivity.                      |
| **Enhanced User Experiences**  | Enables developers to create sophisticated AI systems and context-aware applications, delivering human-like interactions with AI.                                   |
| **Future-Proof Investments**   | LangChain stays up-to-date with the latest advancements in AI and LLMs, ensuring applications remain future-proof and adaptable.                                      |




#Install Dependencies

In [ ]:
!pip install -qU  langchain-google-genai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.6 MB/s eta 0:00:00


#Load Secrets

In [ ]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"]=userdata.get('GOOGLE_API_KEY')
os.environ["GROQ_API_KEY"]=userdata.get('GROQ_API_KEY')

#Loading Prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

# Loading Model

In [ ]:
# Ensure your VertexAI credentials are configured
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05")


In [ ]:
response=llm.invoke(prompt_template.format_prompt(language="French", text="I love programming."))

In [ ]:
response.content

'The most straightforward and common translation is:\n\n**J\'aime programmer.**\n\nHere are a few other options, with slightly different nuances:\n\n*   **J\'adore programmer.** (This emphasizes a stronger feeling, like "I really love programming.")\n*   **J\'aime la programmation.** (This is more formal and refers to programming as a general concept/field.)\n*    **J\'aime faire de la programmation** (I like to do programming)\n\nThe best option depends on the specific context, but "J\'aime programmer" is generally the most suitable.'

# Formatting Response Leveraging Pypdantic Class

In [ ]:
# prompt: Create a pydantic model to have 2 columns english: original sentence passed and translated text

from pydantic import BaseModel

class Translation(BaseModel):
    english: str
    translated: str


In [ ]:
str_resp=llm.with_structured_output(schema=Translation).invoke(prompt_template.format_prompt(language="French", text="I love programming."))

In [ ]:
str_resp

Translation(english='I love programming.', translated="J'adore programmer")

#Loading into Spark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.getOrCreate()

In [ ]:
df=spark.createDataFrame([str_resp],schema="english string, translated string")
df.show()

+-------------------+------------------+
|            english|        translated|
+-------------------+------------------+
|I love programming.|J'adore programmer|
+-------------------+------------------+



In [ ]:
# Ensure your VertexAI credentials are configured
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05",temperature=0)


In [ ]:
llm.invoke("who is MS Dhoni? respond in not more than 50 words")

AIMessage(content="Mahendra Singh Dhoni (MS Dhoni) is a former Indian international cricketer and captain. Known for his calm demeanor and exceptional wicket-keeping skills, he's considered one of the greatest captains and finishers in the history of the sport, leading India to victory in all three major ICC trophies.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-37e59943-4ba7-48e2-ae52-3f1b0ae58f08-0', usage_metadata={'input_tokens': 15, 'output_tokens': 59, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [ ]:
llm.invoke("who is MS Dhoni? respond in not more than 50 words")

AIMessage(content="Mahendra Singh Dhoni (MS Dhoni) is a former Indian international cricketer and captain. Known for his calm demeanor and exceptional wicket-keeping skills, he's considered one of the greatest captains and finishers in the history of the sport, leading India to victory in all three major ICC trophies.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a2afc1d0-eae1-4a2d-bf9c-41887e10b247-0', usage_metadata={'input_tokens': 15, 'output_tokens': 59, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

#Change Temperature to higher value for more randomNess

In [ ]:
# Ensure your VertexAI credentials are configured
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05",temperature=1)


In [ ]:
llm.invoke("who is MS Dhoni? respond in not more than 50 words")

AIMessage(content="Mahendra Singh Dhoni (MS Dhoni) is a former Indian international cricketer and captain. Known for his calm demeanor and exceptional wicket-keeping skills, he's considered one of the greatest captains and finishers in limited-overs cricket, leading India to victory in all three major ICC trophies.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ae3cf809-9018-4362-9854-abef06df6071-0', usage_metadata={'input_tokens': 15, 'output_tokens': 58, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})

In [ ]:
llm.invoke("who is MS Dhoni? respond in not more than 50 words")

AIMessage(content="Mahendra Singh Dhoni (MS Dhoni) is a former Indian international cricketer and captain. Known for his calm demeanor and exceptional wicket-keeping skills, he's considered one of the greatest captains and finishers in cricket history, leading India to multiple major ICC tournament victories.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b8c9b124-ec22-42b7-aa5a-faaa0a9a2852-0', usage_metadata={'input_tokens': 15, 'output_tokens': 54, 'total_tokens': 69, 'input_token_details': {'cache_read': 0}})

#Use External Functions with Tool Calling

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

#Chaning Model with 2 clicks using Langchain

In [ ]:
!pip install -U langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Ensure your VertexAI credentials are configured
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0)


In [ ]:
llm.invoke("what is 2*3 and then multiplied by 100")

AIMessage(content='To calculate this, we need to follow the order of operations (PEMDAS):\n\n1. Multiply 2 and 3: 2 * 3 = 6\n2. Multiply 6 by 100: 6 * 100 = 600\n\nSo the final answer is 600.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 47, 'total_tokens': 109, 'completion_time': 0.225454545, 'prompt_time': 0.005015091, 'queue_time': 0.834260455, 'total_time': 0.230469636}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_e669a124b2', 'finish_reason': 'stop', 'logprobs': None}, id='run-adf24485-0883-420d-8ceb-5b03d1047abd-0', usage_metadata={'input_tokens': 47, 'output_tokens': 62, 'total_tokens': 109})

In [ ]:
llm_with_tools=llm.bind_tools([multiply])

In [ ]:
resp=llm_with_tools.invoke("what is 2*3 ")

In [ ]:
resp.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_8fb9',
  'type': 'tool_call'}]

In [ ]:
multiply.invoke(input=resp.tool_calls[0].get("args"))

6

In [ ]:
resp=llm_with_tools.invoke("what is 2*3 and then multiplied by 100")

In [ ]:
resp

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rpd7', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_g4e6', 'function': {'arguments': '{"a": 6, "b": 100}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 226, 'total_tokens': 263, 'completion_time': 0.134545455, 'prompt_time': 0.01579244, 'queue_time': 0.514418023, 'total_time': 0.150337895}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_5f849c5a0b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3cd57629-affd-4852-9b15-5001995f8052-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_rpd7', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 6, 'b': 100}, 'id': 'call_g4e6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 226, 'output_tokens': 37, 'total_tokens': 263})

In [ ]:
resp.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_rpd7',
  'type': 'tool_call'},
 {'name': 'multiply',
  'args': {'a': 6, 'b': 100},
  'id': 'call_g4e6',
  'type': 'tool_call'}]